In [9]:
import os
import glob

In [8]:
os.path.join('E:/s', 'abd', 'abc', 'd')

'E:/s\\abd\\abc\\d'

In [25]:
os.listdir("E:\\Download\\IEMOCAP_full_release_withoutVideos\\IEMOCAP_full_release\\Session1\\sentences\\wav\\Ses01F_impro01\\")

['Ses01F_impro01_F000.wav',
 'Ses01F_impro01_F001.wav',
 'Ses01F_impro01_F002.wav',
 'Ses01F_impro01_F003.wav',
 'Ses01F_impro01_F004.wav',
 'Ses01F_impro01_F005.wav',
 'Ses01F_impro01_F006.wav',
 'Ses01F_impro01_F007.wav',
 'Ses01F_impro01_F008.wav',
 'Ses01F_impro01_F009.wav',
 'Ses01F_impro01_F010.wav',
 'Ses01F_impro01_F011.wav',
 'Ses01F_impro01_F012.wav',
 'Ses01F_impro01_F013.wav',
 'Ses01F_impro01_F014.wav',
 'Ses01F_impro01_F015.wav',
 'Ses01F_impro01_M000.wav',
 'Ses01F_impro01_M001.wav',
 'Ses01F_impro01_M002.wav',
 'Ses01F_impro01_M003.wav',
 'Ses01F_impro01_M004.wav',
 'Ses01F_impro01_M005.wav',
 'Ses01F_impro01_M006.wav',
 'Ses01F_impro01_M007.wav',
 'Ses01F_impro01_M008.wav',
 'Ses01F_impro01_M009.wav',
 'Ses01F_impro01_M010.wav',
 'Ses01F_impro01_M011.wav',
 'Ses01F_impro01_M012.wav',
 'Ses01F_impro01_M013.wav']

In [18]:
"asbdiasf.wav"[:-4]

'asbdiasf'

In [19]:
a = []
b = [1, 2, 3]

In [21]:
a + b + b

[1, 2, 3, 1, 2, 3]

In [9]:
from data.mfcc_extraction import IEMOCAP_Dataset

ROOT = "E:\\Download\\IEMOCAP_full_release_withoutVideos\\IEMOCAP_full_release"
iemocap = IEMOCAP_Dataset(root=ROOT)

In [10]:
iemocap.LABELS

array(['ang', 'dis', 'exc', 'fea', 'fru', 'hap', 'neu', 'oth', 'sad',
       'sur', 'xxx'], dtype='<U3')

In [2]:
iemocap.utterance_paths[0]

'E:\\Download\\IEMOCAP_full_release_withoutVideos\\IEMOCAP_full_release\\Session1\\sentences\\wav\\Ses01F_impro01\\Ses01F_impro01_F000.wav'

In [3]:
len(iemocap.utterance_list)

10039

In [3]:
len(iemocap.label_map.items())

10039

In [9]:
import wave
import numpy as np
import python_speech_features as psf

PATH = iemocap.utterance_paths[1]
with wave.open(PATH, 'r') as f:
    nchannels, sampwidth, framerate, wav_length = f.getparams()[:4]
    data = np.frombuffer(f.readframes(wav_length), dtype=np.short)
    print(data)
    time = np.arange(0,wav_length) * (1.0/framerate)
    print(time)

[ 31 -31 -23 ... -15 -11 -42]
[0.0000000e+00 6.2500000e-05 1.2500000e-04 ... 1.3822500e+00 1.3823125e+00
 1.3823750e+00]


In [10]:
mel_spec = psf.logfbank(data, framerate, nfilt=40)
delta1 = psf.delta(mel_spec, 2)
delta2 = psf.delta(delta1, 2)
display(mel_spec.shape)
display(delta1.shape)
display(delta2.shape)

(137, 40)

(137, 40)

(137, 40)

In [19]:
np.stack((mel_spec, delta1, delta2), axis=1).shape

(137, 3, 40)

In [21]:
'Ses01F_impro01_F000'[:5]

'Ses01'

In [1]:
import numpy as np

a = np.random.randint(5, size=(3,3,3))
b = 1

np.array((b, a), dtype=object)

array([1, array([[[2, 2, 4],
                  [0, 2, 1],
                  [0, 3, 0]],

                 [[1, 2, 0],
                  [2, 0, 3],
                  [0, 4, 3]],

                 [[0, 1, 3],
                  [0, 2, 3],
                  [3, 1, 0]]])], dtype=object)

In [2]:
sample = np.load("E:\\Download\\IEMOCAP_full_release_withoutVideos\\data\\train\\Ses01F_impro01_F000.npy", allow_pickle=True)

In [3]:
sample[1].shape

(194, 3, 40)

In [22]:
np.pad(sample[1], ((0, 106), (0, 0), (0, 0))).shape

(300, 3, 40)

In [5]:
import torch
import torch.nn as nn

sample_data = torch.tensor(sample[1])
sample_data.shape

torch.Size([194, 3, 40])

In [20]:
nn.functional.pad(sample_data, (0, 0, 0, 0, 0, 300 - 194)).shape

torch.Size([300, 3, 40])

In [10]:
import pickle

with open("data/IEMOCAP.pkl", "rb") as f:

    data = pickle.load(f)

In [11]:
data[0].shape

(1200, 300, 40, 3)

In [12]:
for i in range(len(data)):
    
    print(data[i].shape)

(1200, 300, 40, 3)
(1200, 1)
(420, 300, 40, 3)
(259, 1)
(436, 300, 40, 3)
(298, 1)
(436, 1)
(420, 1)
(259,)
(298,)


In [19]:
data[0][:10].shape

(10, 300, 40, 3)

In [16]:
data[1].shape

(1200, 1)

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class acrnn(nn.Module):
    def __init__(self, num_classes=4, is_training=True,
                 L1=128, L2=256, cell_units=128, num_linear=768,
                 p=10, time_step=150, F1=64, dropout_keep_prob=1):
        super(acrnn, self).__init__()

        self.num_classes = num_classes
        self.is_training = is_training
        self.L1 = L1
        self.L2 = L2
        self.cell_units = cell_units
        self.num_linear = num_linear
        self.p = p
        self.time_step = time_step
        self.F1 = F1
        self.dropout_prob = 1 - dropout_keep_prob

        # tf filter : [filter_height, filter_width, in_channels, out_channels]
        self.conv1 = nn.Conv2d(3, self.L1, (5, 3), padding=(2, 1))       # [5, 3,   3, 128]  
        self.conv2 = nn.Conv2d(self.L1, self.L2, (5, 3), padding=(2, 1)) # [5, 3, 128, 256]
        self.conv3 = nn.Conv2d(self.L2, self.L2, (5, 3), padding=(2, 1)) # [5, 3, 256, 256]
        self.conv4 = nn.Conv2d(self.L2, self.L2, (5, 3), padding=(2, 1)) # [5, 3, 256, 256]
        self.conv5 = nn.Conv2d(self.L2, self.L2, (5, 3), padding=(2, 1)) # [5, 3, 128, 256]
        self.conv6 = nn.Conv2d(self.L2, self.L2, (5, 3), padding=(2, 1)) # [5, 3, 128, 256]

        self.linear1 = nn.Linear(self.p*self.L2, self.num_linear) # [10*256, 768]
        self.bn = nn.BatchNorm1d(self.num_linear)

        self.relu = nn.LeakyReLU(0.01)
        self.dropout = nn.Dropout2d(p=self.dropout_prob)
        
        self.rnn = nn.LSTM(input_size=self.num_linear, hidden_size=self.cell_units, 
                            batch_first=True, num_layers=1, bidirectional=True) 

        # for attention
        self.a_fc1 = nn.Linear(2*self.cell_units, 1)  
        self.a_fc2 = nn.Linear(1, 1)
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)

        # fully connected layers
        self.fc1 = nn.Linear(2*self.cell_units, self.F1) # [2*128, 64]
        self.fc2 = nn.Linear(self.F1, self.num_classes) # [num_classes]

    
    def forward(self, x):
        
        layer1 = self.relu(self.conv1(x))
        layer1 = F.max_pool2d(layer1, kernel_size=(2, 4), stride=(2, 4))   # [1,2,4,1], padding = 'valid'
        layer1 = self.dropout(layer1)

        layer2 = self.relu(self.conv2(layer1))
        layer2 = self.dropout(layer2)
        
        layer3 = self.relu(self.conv3(layer2))
        layer3 = self.dropout(layer3)

        layer4 = self.relu(self.conv4(layer3))
        layer4 = self.dropout(layer4)

        layer5 = self.relu(self.conv5(layer4))
        layer5 = self.dropout(layer5)

        layer6 = self.relu(self.conv6(layer5))
        layer6 = self.dropout(layer6)
        
        # lstm
        layer6 = layer6.permute(0, 2, 3, 1)
        layer6 = layer6.reshape(-1, self.time_step, self.L2*self.p)        # (-1, 150, 256*10)
        layer6 = layer6.reshape(-1, self.L2*self.p)                        # (1500, 2560)

        linear1 = self.relu(self.bn(self.linear1(layer6)))                 # [1500, 768]
        linear1 = linear1.reshape(-1, self.time_step, self.num_linear)     # [10, 150, 768]
        
        outputs1, output_states1 = self.rnn(linear1)                       # outputs1 : [10, 150, 128] (B,T,D)

        # # attention
        v = self.sigmoid(self.a_fc1(outputs1))                  # (10, 150, 1)
        alphas = self.softmax(self.a_fc2(v).squeeze())          # (B,T) shape, alphas are attention weights
        gru = (alphas.unsqueeze(2) * outputs1).sum(axis=1)      # (B,D)
        
        # # fc
        fully1 = self.relu(self.fc1(gru))
        fully1 = self.dropout(fully1)
        Ylogits = self.fc2(fully1)
        Ylogits = self.softmax(Ylogits)

        return Ylogits


In [50]:
import numpy as np
# data_194 = torch.randint(0, 5, size=(10, 194, 40, 3))
# data_194.transpose((0, 3, 1, 2))
data_194 = torch.tensor(np.random.randint(3, size=(10, 300, 40, 3)).transpose((0, 3, 1, 2))).float()
data_194.shape

torch.Size([10, 3, 300, 40])

In [2]:
from dataset.datasets import IEMOCAPSamples
from torch.utils.data import DataLoader

PATH = "data/data_tiny"
iemocap_samples = IEMOCAPSamples(PATH)
iemocap_loader = DataLoader(iemocap_samples, batch_size=5)

In [2]:
len(iemocap_loader)

6

In [3]:
X, y = next(iter(iemocap_loader))

In [4]:
X.permute((0, 2, 1, 3)).shape

torch.Size([5, 3, 1500, 40])

In [9]:
model = acrnn(num_classes=11)
y_preds = model(X.permute((0, 2, 1, 3)).float())

RuntimeError: mat1 and mat2 shapes cannot be multiplied (5x1920000 and 2560x768)

In [7]:
y_preds

tensor([[0.0847, 0.0826, 0.0869, 0.0958, 0.0890, 0.0883, 0.0821, 0.0870, 0.0862,
         0.1093, 0.1080],
        [0.0887, 0.0778, 0.0914, 0.0964, 0.0899, 0.0919, 0.0789, 0.0907, 0.0810,
         0.1100, 0.1034],
        [0.0887, 0.0778, 0.0914, 0.0964, 0.0899, 0.0919, 0.0789, 0.0907, 0.0810,
         0.1100, 0.1034],
        [0.0887, 0.0778, 0.0914, 0.0964, 0.0899, 0.0919, 0.0789, 0.0907, 0.0810,
         0.1100, 0.1034],
        [0.0887, 0.0778, 0.0914, 0.0963, 0.0899, 0.0919, 0.0788, 0.0906, 0.0810,
         0.1100, 0.1035],
        [0.0851, 0.0876, 0.0801, 0.0962, 0.0843, 0.0858, 0.0906, 0.0870, 0.0914,
         0.1037, 0.1082],
        [0.0852, 0.0827, 0.0860, 0.0958, 0.0892, 0.0890, 0.0823, 0.0869, 0.0865,
         0.1084, 0.1080],
        [0.0887, 0.0778, 0.0914, 0.0964, 0.0899, 0.0919, 0.0789, 0.0907, 0.0810,
         0.1100, 0.1034],
        [0.0887, 0.0778, 0.0914, 0.0964, 0.0899, 0.0919, 0.0789, 0.0907, 0.0810,
         0.1100, 0.1034],
        [0.0887, 0.0778, 0.0914, 0.09

In [16]:
X

tensor([[[[ 9.3616e+00,  9.5302e+00,  1.0740e+01,  ...,  1.1792e+01,
            1.0565e+01,  1.0394e+01],
          [-3.6884e-01, -3.7800e-01, -1.0049e-01,  ..., -2.3690e-01,
           -5.6242e-01, -1.3496e+00],
          [ 1.8911e-01,  9.4106e-02,  8.4427e-02,  ..., -5.3681e-02,
            8.4809e-03,  1.8676e-01]],

         [[ 8.4326e+00,  9.7718e+00,  1.0101e+01,  ...,  1.1353e+01,
            8.8876e+00,  8.1949e+00],
          [ 1.3032e-01, -2.5628e-01,  2.5058e-01,  ..., -3.6818e-01,
           -4.8817e-01, -8.9748e-01],
          [ 2.0864e-01,  1.8645e-01,  6.1018e-02,  ..., -5.2286e-02,
            4.3677e-02,  3.3841e-01]],

         [[ 7.9819e+00,  7.5194e+00,  1.0557e+01,  ...,  1.0827e+01,
            8.5921e+00,  4.7452e+00],
          [ 3.2712e-01,  3.1668e-02,  1.4610e-01,  ..., -4.3967e-01,
           -5.5715e-01, -6.4187e-01],
          [ 1.0717e-01,  2.7920e-01, -1.7041e-02,  ..., -8.7261e-04,
            7.5904e-02,  4.1618e-01]],

         ...,

         [[ 0.00

In [2]:
import numpy as np
a = np.array([[1], [1], [2]])
a.reshape(-1)

array([1, 1, 2])